## Physical Analysis

#### Goal
- Derive the energy of the incoming particle by analyzing the amount of energy that it deposited

- Relevant Information
    - Electron-hole pair creation energy in silicon is 3.66 +/- 0.03 eV (independent of incoming photon energy)
        - "Mean energy required to produce an electron-hole pair in silicon for photons of energies between 50 and 1500 eV", Scholze et al. 1998
        - <a href="https://pdfs.semanticscholar.org/32a7/3c7b1a881d6992650390f9c97540529fdc7e.pdf?_ga=2.222678866.858335769.1593874624-1310422571.1590444937">PDF of the paper</a>
        - There is a temperature dependence, but the fractional change in the creation energy is of order ~1% over our range of interest
        

In [ ]:
%matplotlib widget
import glob
import os
import sys
import astropy.constants as physical_constants
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from astropy.visualization import ImageNormalize, SqrtStretch, LogStretch, LinearStretch, ZScaleInterval, ManualInterval
import astropy.units as u
import dask.array as da
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1 import make_axes_locatable
plt.style.use('ggplot')
import numpy as np
import pylandau 

#### Estimating the average particle length using stats from Table 4

- STIS
    - Average CR affects 7 pixels
- ACS/HRC
    - Average CR affects 7 pixels
- ACS/WFC
    - Average CR affects 9 pixels
- WFC3/UVIS
    - Average CR affects 9 pixels

In [ ]:
gamma = lambda v: 1/np.sqrt(1-(v/physical_constants.c)**2)
relativistic_momentum = lambda v,m: gamma(v) * m * v 
relativistic_energy = lambda v,m: gamma(v)*m*physical_constants.c**2
relativistic_kinetic_energy = lambda v,m: (gamma(v) - 1)*m*physical_constants.c**2

def compute_Wmax(m, v):
    m_e = physical_constants.m_e
    c = physical_constants.c
    numerator = 2 * m_e * c**2 * (v/c)**2 * gamma(v)**2
    denominator = 1 + 2 * gamma(v) * m_e/m + (m_e/m)**2
    return numerator/denominator

def bethe_bloch(m, v, z, Z, A, W_max, I):
    K = 0.307075 * u.megaelectronvolt * u.cm**2 / u.mol
    m_e = physical_constants.m_e
    c = physical_constants.c
    constant = K * z**2 * Z/A * 1/(v/c)**2
    density_corr = 0
    scaling = 0.5 * np.log((2*m_e*c**2*(v/c)**2 * gamma(v)**2 * W_max)/I**2) - (v/c)**2 - density_corr/2
    de_dx = constant * scaling
    return de_dx

def sternheimer_density_corr(beta, lorentz_factor):
    X = np.log10(beta.value*lorentz_factor.value)
    X0 = 0.966
    X1 = 2.5
    a = 0.3755
    m = 3.233
    C = -4.435
    dcorr = []
    try:
        for val in X:
            if X0 < val and val < X1:
                delta = 4.6052*val + a*(X1 - val)**m + C
            elif val >= X1:
                delta = 4.6052*val + C
            elif val < X0:
                delta = 0.0865*10**(2*(val - X0))
            dcorr.append(delta)
    except TypeError:
        if X0 < X and X < X1:
                delta = 4.6052*X + a*(X1 - X)**m + C
        elif X >= X1:
            delta = 4.6052*X + C
        elif X < X0:
            delta = 0.0865*10**(2*(X - X0))
        return delta
    else:
        return dcorr
    
def landau(m, v, z, Z, A, I, dx):
    c = physical_constants.c
    K = 0.307075 * u.megaelectronvolt * u.cm**2 / u.mol
    m_e = physical_constants.m_e
    constant = K/2 * Z/A * z**2 * dx * 1/(v/c)**2
#     density_corr = sternheimer_density_corr(v/c, gamma(v))
    density_corr = 0
    scaling = np.log((2 * m_e * c**2 * (v/c)**2 * gamma(v)**2)/I) + np.log(constant/I) + 0.2 - (v/c)**2 - density_corr 
    return constant * scaling

In [ ]:
dx = 3
dy = 3

In [ ]:
avg_proj_path = lambda dx, dy, psize: np.sqrt(dx**2 + dy**2) * psize 

In [ ]:
avg_stis = avg_proj_path(3, 3, 21)
avg_wfc = avg_proj_path(4, 4, 15)

In [ ]:
avg_stis

In [ ]:
si_eh_pair = 3.71 *u.eV / u.electron

In [ ]:
rdnse = 4 *u.electron
rdnse_eV = rdnse * si_eh_pair

In [ ]:
dc = 0.01 *u.electron/u.second
dc_err_1000s = np.sqrt(dc * 1000 *u.second * si_eh_pair)

In [ ]:
dc_err_1000s

In [ ]:
rdnse_eV

In [ ]:
total_uncertainty = np.sqrt(dc_err_1000s.value**2 + rdnse_eV.value**2) * u.eV

In [ ]:
total_uncertainty.to('keV')

In [ ]:
np.sqrt((10*21)**2 + 14**2)

In [ ]:
base_path = os.path.join(os.path.dirname(os.getcwd()))
pipeline_path = os.path.join(os.path.dirname(os.getcwd()), 'pipeline/')
sys.path.append(pipeline_path)

In [ ]:
pipeline_path

Examining the relativistic limit for protons

In [ ]:
m_p = physical_constants.m_p

In [ ]:
0.8*physical_constants.c

In [ ]:
velocities = np.linspace(0.25*physical_constants.c.value,0.99999999*physical_constants.c.value, 10000) * u.m/u.s
lorentz_factor = np.array([gamma(v).value for v in velocities])
spatial_momentum = np.array([relativistic_momentum(v, m_p).value for v in velocities]) * u.kg * u.m/u.s 
energy = np.array([relativistic_energy(v, m_p).value for v in velocities]) * u.kg * u.m**2 /u.s**2 # joules

In [ ]:
gamma(0.71*physical_constants.c) * 0.71

In [ ]:
beta = velocities/physical_constants.c
lorentz_factor = gamma(velocities)
product = beta*lorentz_factor

In [ ]:
product[np.isclose(product.value, 2.1, atol=5e-5)]

### Computing the conditions for Landau to apply
t = 290 $\mu$m

k = $\xi/W_{max}$<< 1

$E_\text{K}$ << $\xi$

In [ ]:
si_density = 2.329 * u.g/u.cm**3

In [ ]:
z=1
Z=14
A=28.0855*u.g/u.mol
beta_val=0.7
K = 0.307075 * u.megaelectronvolt * u.cm**2 / u.mol
m_e = physical_constants.m_e
xi_betasq = K/2 * Z/A * z**2 *  si_density
xi_betasq = xi_betasq.to('keV/micrometer')

In [ ]:
xi_betasq

In [ ]:
beta_bichsel1 = beta[np.isclose(product.value, 2.1, atol=5e-5)][0]

In [ ]:
beta_bichsel2 = beta[np.isclose(product.value, 8.5, atol=2e-2)][0]

In [ ]:
beta_bichsel1, beta_bichsel2

In [ ]:
xi1 = (xi_betasq / beta_bichsel1**2) * 290*u.micrometer
xi2 = (xi_betasq / beta_bichsel2**2) * 290*u.micrometer

In [ ]:
xi1, xi2

In [ ]:
Wmax1 = compute_Wmax(m_p, beta_bichsel1*physical_constants.c).to('keV')
Wmax2 = compute_Wmax(m_p, beta_bichsel2*physical_constants.c).to('keV')

In [ ]:
k1 = xi1/Wmax1
k2 = xi2/Wmax2

In [ ]:
k1

In [ ]:
print(f"{k1:.2e}, {k2:.2e}")

In [ ]:
KE_1 = relativistic_kinetic_energy(beta_bichsel1*physical_constants.c, m_p).to('GeV')

In [ ]:
KE_1

In [ ]:
KE_2 = relativistic_kinetic_energy(beta_bichsel2*physical_constants.c, m_p).to('GeV')

In [ ]:
KE_2

#### Assume proton has a kinetic energy of 4 GeV

compute gamma and beta


In [ ]:
gamma_4gev = 4/0.938 + 1
beta_4gev = np.sqrt(1 - 1/gamma_4gev**2)

In [ ]:
gamma_4gev, beta_4gev

In [ ]:
beta

In [ ]:
wmax = compute_Wmax(m_p, beta_4gev * physical_constants.c).to('keV')

In [ ]:
f"{wmax:.2e}"

In [ ]:
xi_betasq * 290*u.micrometer

In [ ]:
k = xi_betasq * 290*u.micrometer / wmax

In [ ]:
print(f"{k:.2e}")

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.semilogy(beta, product, marker='o', ms=2.)
ax.axhline(2, ls='--')
ax.axhline(8.5,ls='--')
ax.set_xlabel(r'$\beta$')
ax.set_ylabel(r'$\beta\gamma$')

#### Convert the energy to MeV

In [ ]:
energy_MeV = energy.to('Joule').to('megaelectronvolt')

#### Calculate the Kinetic Energy

In [ ]:
kinetc_energy = energy_MeV - m_p*physical_constants.c**2

#### Convert the momentum to MeV/c

In [ ]:
spatial_momentum_MeV_per_c = spatial_momentum * physical_constants.c
spatial_momentum_MeV_per_c = spatial_momentum_MeV_per_c.to('Joule').to('megaelectronvolt')

In [ ]:
spatial_momentum_MeV_per_c

In [ ]:
beta_lorentz = velocities/physical_constants.c * lorentz_factor

In [ ]:
ms=5
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(10,4.5), gridspec_kw={'wspace':0.5})

ax[0].scatter(beta_lorentz, lorentz_factor,s=ms)
ax[1].scatter(beta_lorentz, spatial_momentum_MeV_per_c,s=ms)
ax[2].scatter(beta_lorentz, energy_MeV,s=ms)
for a in ax:
    a.set_xlabel(r'$\beta\gamma$')
    a.set_yscale('log')
    a.set_xscale('log')
    
ax[0].set_ylabel('Lorentz Factor [dimensionless]')
ax[1].set_ylabel(f"Relativistic Momentum [{spatial_momentum_MeV_per_c.unit}/c]")
ax[2].set_ylabel(f"Relativistic Energy [{energy_MeV.unit}]")


In [ ]:
wmax = compute_Wmax(m_p, velocities)

In [ ]:
wmax = wmax.to('Joule').to('keV')

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.loglog(kinetc_energy, wmax)
ax.set_xlabel(f'Kinetic Energy [{kinetc_energy.unit}]')
ax.set_ylabel('Wmax'+f' [{wmax.unit}]')

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.semilogy(product, kinetc_energy)
ax.set_ylabel(f'Kinetic Energy [{kinetc_energy.unit}]')
ax.set_xlabel(r'$\beta\gamma$')
ax.set_xscale('log')
ax.axvline(2.1)
ax.axvline(8.5)

In [ ]:
def sternheimer_density_corr(beta, lorentz_factor):
    X = np.log10(beta.value*lorentz_factor.value)
    X0 = 0.966
    X1 = 2.5
    a = 0.3755
    m = 3.233
    C = -4.435
    dcorr = []
    try:
        for val in X:
            if X0 < val and val < X1:
                delta = 4.6052*val + a*(X1 - val)**m + C
            elif val >= X1:
                delta = 4.6052*val + C
            elif val < X0:
                delta = 0.0865*10**(2*(val - X0))
            dcorr.append(delta)
    except TypeError:
        if X0 < X and X < X1:
                delta = 4.6052*X + a*(X1 - X)**m + C
        elif X >= X1:
            delta = 4.6052*X + C
        elif X < X0:
            delta = 0.0865*10**(2*(X - X0))
        return delta
    else:
        return dcorr
    

In [ ]:
def landau(m, v, z, Z, A, I, dx):
    c = physical_constants.c
    K = 0.307075 * u.megaelectronvolt * u.cm**2 / u.mol
    m_e = physical_constants.m_e
    constant = K/2 * Z/A * z**2 * dx * 1/(v/c)**2
#     density_corr = sternheimer_density_corr(v/c, gamma(v))
    density_corr = 0
    scaling = np.log((2 * m_e * c**2 * (v/c)**2 * gamma(v)**2)/I) + np.log(constant/I) + 0.2 - (v/c)**2 - density_corr 
    return constant * scaling

In [ ]:
pb_density = 11.34 * u.g/u.cm**3
si_density = 2.329 * u.g/u.cm**3

## Checking $k$

In [ ]:
v = 0.6*physical_constants.c
wmax_test = compute_Wmax(m_p, v)
wmax_test=wmax_test.to('keV')
wmax_test

In [ ]:
z=1
Z=14
A=28.0855*u.g/u.mol
beta_val=0.7
K = 0.307075 * u.megaelectronvolt * u.cm**2 / u.mol
m_e = physical_constants.m_e
xi = K/2 * Z/A * z**2 *  1/(beta)**2 * si_density
xi = xi.to('keV/micrometer')
xi *= 290 *u.micrometer

In [ ]:
xi 

In [ ]:
xi/wmax_test

In [ ]:
si_de_dx = bethe_bloch(m_p, velocities, z=1, Z=14, A=28.0855*u.g/u.mol, W_max=wmax, I=173*u.eV)
pb_de_dx = bethe_bloch(m_p, velocities, z=1, Z=82, A=207.2*u.g/u.mol, W_max=wmax, I=823*u.eV)


In [ ]:
si_landau_de_100 = landau(m_p, velocities, z=1, Z=14, A=28.0855*u.g/u.mol, I=173*u.eV, dx=200*u.micrometer * si_density)
si_landau_de_70 = landau(m_p, velocities, z=1, Z=14, A=28.0855*u.g/u.mol, I=173*u.eV, dx=100*u.micrometer * si_density)

In [ ]:
speed = 0.999*physical_constants.c

In [ ]:
speed_12_GeV = 0.99694*physical_constants.c

In [ ]:
mpv = landau(m_p, speed,z=1, Z=14, A=28.0855*u.g/u.mol, I=173*u.eV, dx=100*u.micrometer * si_density)

In [ ]:
mpv.to('keV')

In [ ]:
si_landau_de_100 = si_landau_de_100.to('keV')
si_landau_de_70 = si_landau_de_70.to('keV')

In [ ]:
kinetc_energy = energy_MeV - m_p*physical_constants.c**2

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6,5))
ax.semilogy(beta, kinetc_energy, label='')

In [ ]:
relativistic_energy(0.99694*physical_constants.c, m_p).to('GeV')

In [ ]:
kinetc_energy[408]

In [ ]:
si_de_dx[408]

In [ ]:
pb_loss_per_x = (pb_de_dx*pb_density).to('MeV/micrometer')
si_loss_per_x = (si_de_dx*si_density).to('MeV/micrometer')

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6,5))
ax.semilogx(kinetc_energy, si_de_dx, label='Si')
ax.semilogx(kinetc_energy, pb_de_dx, label='Pb')
ax.set_xlim(5e1, 8e6)
# ax.set_xscale('log')
# ax.set_yscale('log')
ax.set_ylim(1,10)
ax.set_xlabel('Kinetic Energy MeV')
ax.set_ylabel(f"Linear Stopping Power [{si_de_dx.unit}]")
ax.legend(loc='upper right', edgecolor='k')
ax.set_title('Proton Stopping Power')

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6,5))
ax.semilogx(kinetc_energy, si_loss_per_x, label='Si')
ax.semilogx(kinetc_energy, pb_loss_per_x, label='Pb')
ax.set_xlim(5e1, 8e6)
# ax.set_xscale('log')
# ax.set_yscale('log')
ax.set_ylim(0,0.01)
ax.set_xlabel('Kinetic Energy MeV')
ax.set_ylabel(f"Energy Loss [{si_loss_per_x.unit}]")
ax.legend(loc='upper right', edgecolor='k')
ax.set_title('Proton Energy Loss')

In [ ]:
thickness = 1 * u.micrometer
total_loss_min = (si_loss_per_x * 280*thickness).to('keV')
total_loss_avg = (si_loss_per_x * 290 * thickness ).to('keV')
total_loss_max = (si_loss_per_x* 300 *thickness).to('keV')

In [ ]:
si_loss_per_x

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6,5))

ax.fill_between(kinetc_energy, total_loss_min, total_loss_max, alpha=0.5)

ax.semilogx(kinetc_energy, total_loss_min,ls='--', lw=2.25, c='r', label=f'{280*thickness}')
ax.semilogx(kinetc_energy, total_loss_avg, label=f'{200*thickness}')
ax.semilogx(kinetc_energy, total_loss_max,ls=':', c='r', lw=2.25, label=f'{300*thickness}')
ax.set_xlim(5e1, 8e6)
# ax.set_xscale('log')
# ax.set_yscale('log')
ax.set_ylim(0,250)
ax.set_xlabel('Kinetic Energy MeV')
ax.set_ylabel(f"Energy Loss [{total_loss_avg.unit}]")
ax.legend(loc='upper right', edgecolor='k')
ax.set_title('Proton Energy Loss for Si (Bethe-Bloch)')

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
for i in [200, 250]:
    energy_loss = landau(m_p, velocities, z=1, Z=14, A=28.0855*u.g/u.mol, I=173*u.eV, dx=i*u.micrometer * si_density)
    energy_loss = energy_loss.to('keV')
    if i == 280:
        c = 'tab:purple'
    else:
        c = 'tab:red'
    ax.semilogx(kinetc_energy, energy_loss,c=c, label=f'Landau pred. for {i:.0f}$\mu$m')
#     ax.semilogx(kinetc_energy, (si_loss_per_x * i*u.micrometer).to('keV'),c=c, ls='--',label=f'Bethe pred. for {i:.0f}$\mu$m')
#     ax.semilogx(kinetc_energy, si_landau_de_70, label='Losses for 70$\mu$m')
ax.set_xlabel(f"Kinetic Energy [{kinetc_energy.unit}]")
ax.set_ylabel(f"Energy Loss [{energy_loss.unit}]")
ax.legend(loc='best', edgecolor='k')
ax.set_ylim(20, 150)
ax.set_xlim(100, 1e6)
ax.yaxis.set_major_locator(plt.MultipleLocator(20))
ax.yaxis.set_minor_locator(plt.MultipleLocator(5))
ax.set_title('Energy Loss Predictions')
fig.savefig('/user/nmiles/landau_vs_bethe.jpeg',format='jpeg',dpi=250, bbox_inches='tight')

In [ ]:
plt.close('all')

## Gyroradius in Earth's Magnetosphere

In [ ]:
import pmagpy.ipmag as ipmag
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [ ]:
altitude = np.linspace(100, 1000, 10)
date = 2002
lat = 0
lon = 0
# lat_grid, lon_grid = np.meshgrid(lat, lon)
# coordinates = list(zip(lat_grid.ravel(), lon_grid.ravel()))
B_strength = []
for a in altitude:
    b_field = ipmag.igrf([date, a, lat, lon])
    B_strength.append(b_field[-1])
# B_strength_grid = np.array(B_strength).reshape(lat_grid.shape)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.scatter(altitude, B_strength)

First we generate the B field strength in HST's orbital path with 1 degree spatial resolution

In [ ]:
altitude = 1200
date = 2002
lat = np.linspace(-30, 30, 1 * 60 + 1)
lon = np.linspace(0, 360, 1 * 60 + 1)
lat_grid, lon_grid = np.meshgrid(lat, lon)
coordinates = list(zip(lat_grid.ravel(), lon_grid.ravel()))
B_strength = []
for coords in coordinates:
    b_field = ipmag.igrf([date, altitude, coords[0], coords[1]])
    B_strength.append(b_field[-1])
B_strength_grid = np.array(B_strength).reshape(lat_grid.shape)

In [ ]:
len(lat)

In [ ]:
len(lon)

In [ ]:
B_strength = B_strength * u.nT

In [ ]:
B_strength[1]

### Computing kinetic energy from gyromagnetic rigidity

In [ ]:
rigidity = 5 * u.gigavolt

In [ ]:
momentum = rigidity * 1.602e-19 * u.C

In [ ]:
total_E = np.sqrt((m_p*physical_constants.c**2)**2 + momentum.to('GeV')**2).to('GeV')

In [ ]:
total_E

In [ ]:
kinetic_energy = total_E - m_p * physical_constants.c**2

In [ ]:
kinetic_energy

In [ ]:
def relativistic_gyroradius(m,v,B,q):
    R = gamma(v) * m * v/(q*B)
    return R.to('m').to('km')
    

In [ ]:
physical_constants.e

In [ ]:
gyro_radii = []
for B in B_strength:
    R = relativistic_gyroradius(m=physical_constants.m_p, v=0.9*physical_constants.c, B=B.to('T'), q=1.602e-19 *u.C)
    gyro_radii.append(R.value)

In [ ]:
B_strength

In [ ]:
gyro_radii = gyro_radii * u.km

In [ ]:
fname = '/ifs/missions/projects/plcosmic/hst_cosmic_rays/APJ_plots/HYP_50M_SR_W.tif'
fig, ax = plt.subplots(nrows=1,
                       ncols=1,
                       figsize=(8, 5),
                       subplot_kw={'projection': ccrs.PlateCarree()}
                      )

crs = ccrs.PlateCarree()
transform = crs._as_mpl_transform(ax)
ax.imshow(
    plt.imread(fname),
    origin='upper',
    transform=crs,
    extent=[-180, 180, -30, 30]
)
# ax.stock_img()
ax.coastlines()
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=1, color='k', alpha=0.4, linestyle='--')
gl.xlabels_top = False
gl.ylabels_left = True
gl.ylabels_right = False
gl.xlines = True
# gl.xlocator = mticker.FixedLocator([-180, -45, 0, 45, 180])
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlocator = plt.MultipleLocator(30)
gl.ylocator = plt.MultipleLocator(20)
gl.xlabel_style = {'size': 10, 'color': 'black'}
gl.xlabel_style = {'color': 'black'}

scat = ax.scatter(
    lon, 
    lat,
    marker='o',
    s=5.5,
    c='k', alpha=0.9,
    transform=ccrs.PlateCarree()
            )

cntr = ax.contour(
    lon_grid,
    lat_grid,
    B_strength_grid,
    cmap='plasma', 
    levels=14,
    alpha=1, 
    lw=2, 
    transform=ccrs.PlateCarree()
    )

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.scatter(B_strength, gyro_radii, s=3)
ax.set_ylabel(f'Gyroradius [{gyro_radii.unit}]')
ax.set_xlabel(f'|B| {B_strength.unit}')

In [ ]:
fname = '/ifs/missions/projects/plcosmic/hst_cosmic_rays/APJ_plots/HYP_50M_SR_W.tif'
fig, ax = plt.subplots(nrows=1,
                       ncols=1,
                       figsize=(8, 5),
                       subplot_kw={'projection': ccrs.PlateCarree()}
                      )

crs = ccrs.PlateCarree()
transform = crs._as_mpl_transform(ax)
ax.imshow(
    plt.imread(fname),
    origin='upper',
    transform=crs,
    extent=[-180, 180, -90, 90]
)
# ax.stock_img()
ax.coastlines()
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=1, color='k', alpha=0.4, linestyle='--')
gl.xlabels_top = False
gl.ylabels_left = True
gl.ylabels_right = False
gl.xlines = True
# gl.xlocator = mticker.FixedLocator([-180, -45, 0, 45, 180])
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlocator = plt.MultipleLocator(30)
gl.ylocator = plt.MultipleLocator(20)
gl.xlabel_style = {'size': 10, 'color': 'black'}
gl.xlabel_style = {'color': 'black'}

scat = ax.scatter(
    lon, 
    lat,
    marker='o',
    s=3.5,
    c='k', alpha=0.7,
    transform=ccrs.PlateCarree()
            )

# cntr = ax.contour(
#     lon_grid,
#     lat_grid,
#     B_strength_grid,
#     cmap='plasma', 
#     levels=10,
#     alpha=1, 
#     lw=2, 
#     transform=ccrs.PlateCarree()
#     )

In [ ]:
lon - 180

In [ ]:
lat